# ECS713: week 6 Lab Sheet

This lab sheet covers 
- IO

## Learning Objectives

By the time you complete this sheet you should be able to
- understand how to declare a new typeclass
- how to make a type an instance of a typeclass
- recognise some common typeclasses and the functions they provide
- understand the use of the deriving connective

## Turn off the annoying linter

Run the cell below to turn off the annoying linter, which suggests improvements to your code that aren't appropriate for these exercises. 

In [ ]:
:opt no-lint

## Task 1. Basic IO

**Problems with Jupyter** There are problems with using Jupyter notebooks in connection with IO. The basic issue is that standard Haskell IO functions are fairly simple-minded, and the framework you are likely to be using to run the notebooks is not. If you are doing what I am, then you are running the server on a Docker container. This means in particular that it has an encapsulated filesystem. So the starting point is that Haskell expects interactions to be taking place in the container. This is a particular issue for inputs. The notebook will look for files in the container's filesystem, not the one on your computer. This is not so bad, you can always upload things. But it is a real issue for keyboard input. It will try to take input from a handle called "stdin". This is not connected to your keyboard, so it will appear empty. Here is a simple experiment, run the cell below: 

In [14]:
getLine

: 

You should get something like: 

```<stdin>: hGetLine: end of file```

This basically says that the notebook is not capable of getting input in this form. The best way to get round that is to install Haskell (better install stack which we will be using later), and then use the interpreter: ghci. We are shortly going to move off Jupyter notebooks because we want to be able to access non-Haskell systems such as databases. 

However, output is OK. Haskell sends its results on "stdout", so this is connected up appropriately, and you may not even be able to see the critical difference between the notebook printing out the result of a calculation, and something being printed as a side effect during a calculation. 

In [1]:
-- String printed out as result
"Hi there!"

"Hi there!"

In [2]:
-- String printed out during IO operation. 
putStrLn "Hi there!"

Hi there!

Basic IO is documented at: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:25

The simple functions can be found at: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:26

For the purpose of these exercises we will mainly be using file-based IO: https://hackage.haskell.org/package/base-4.14.0.0/docs/Prelude.html#g:29

In particular we will want: 

```type FilePath = String
readFile :: FilePath -> IO String
writeFile :: FilePath -> String -> IO ()```


We have a file "poem.txt", to access it: 

In [3]:
readFile "poem.txt"

"Twas brillig, and the slithy toves\nDid gyre and gimble in the wabe:\nAll mimsy were the borogoves,\nAnd the mome raths outgrabe.\n\n\"Beware the Jabberwock, my son!\nThe jaws that bite, the claws that catch!\nBeware the Jubjub bird, and shun\nThe frumious Bandersnatch!\"\n\nHe took his vorpal sword in hand;\nLong time the manxome foe he sought-\nSo rested he by the Tumtum tree\nAnd stood awhile in thought.\n\nAnd, as in uffish thought he stood,\nThe Jabberwock, with eyes of flame,\nCame whiffling through the tulgey wood,\nAnd burbled as it came!\n\nOne, two! One, two! And through and through\nThe vorpal blade went snicker-snack!\nHe left it dead, and with its head\nHe went galumphing back.\n\n\"And hast thou slain the Jabberwock?\nCome to my arms, my beamish boy!\nO frabjous day! Callooh! Callay!\"\nHe chortled in his joy.\n\n'Twas brillig, and the slithy toves\nDid gyre and gimble in the wabe:\nAll mimsy were the borogoves,\nAnd the mome raths outgrabe.\n      "

We can write definitions: 

In [4]:
ioPoem = readFile "poem.txt"
poem <- readFile "poem.txt"
:t ioPoem
:t poem

ioPoem :: IO String

poem :: String

The `<-` form only works in the context of a `do` block, which the Jupyter cells implicitly are. 

`ioPoem` is an `IO String`, not a `String`:

In [5]:
length ioPoem

: 

But `poem` is a `String`.

In [6]:
length poem

939

You can do local declarations with IO, but you can't use the `<-` form: 

In [17]:
let poem <- readFile "poem.txt" in length poem

: 

Let's import the `Data.Char` module so we have a function to map characters to upper case: 

In [19]:
import Data.Char

And use it as in lectures to map everything to upper case: 

In [20]:
do 
  -- IO read stuff here
  poem <- readFile "poem.txt"
  -- Pure (non-IO) calculations here
  let upperPoem = map toUpper poem
  -- IO write (output) stuff here
  putStr upperPoem

TWAS BRILLIG, AND THE SLITHY TOVES
DID GYRE AND GIMBLE IN THE WABE:
ALL MIMSY WERE THE BOROGOVES,
AND THE MOME RATHS OUTGRABE.

"BEWARE THE JABBERWOCK, MY SON!
THE JAWS THAT BITE, THE CLAWS THAT CATCH!
BEWARE THE JUBJUB BIRD, AND SHUN
THE FRUMIOUS BANDERSNATCH!"

HE TOOK HIS VORPAL SWORD IN HAND;
LONG TIME THE MANXOME FOE HE SOUGHT-
SO RESTED HE BY THE TUMTUM TREE
AND STOOD AWHILE IN THOUGHT.

AND, AS IN UFFISH THOUGHT HE STOOD,
THE JABBERWOCK, WITH EYES OF FLAME,
CAME WHIFFLING THROUGH THE TULGEY WOOD,
AND BURBLED AS IT CAME!

ONE, TWO! ONE, TWO! AND THROUGH AND THROUGH
THE VORPAL BLADE WENT SNICKER-SNACK!
HE LEFT IT DEAD, AND WITH ITS HEAD
HE WENT GALUMPHING BACK.

"AND HAST THOU SLAIN THE JABBERWOCK?
COME TO MY ARMS, MY BEAMISH BOY!
O FRABJOUS DAY! CALLOOH! CALLAY!"
HE CHORTLED IN HIS JOY.

'TWAS BRILLIG, AND THE SLITHY TOVES
DID GYRE AND GIMBLE IN THE WABE:
ALL MIMSY WERE THE BOROGOVES,
AND THE MOME RATHS OUTGRABE.

Haskell is very uniform. We can assign this `do` block a name: 

In [21]:
printUpperPoem = 
  do 
    -- IO read stuff here
    poem <- readFile "poem.txt"
    -- Pure (non-IO) calculations here
    let upperPoem = map toUpper poem
    -- IO write (output) stuff here
    putStr upperPoem

And then we can run it: 

In [ ]:
printUpperPoem

We can extract the filename "poem.txt" and make a function that will convert any file to upper case: 

In [24]:
printUpper filename = 
  do 
    -- IO read stuff here
    poem <- readFile filename
    -- Pure (non-IO) calculations here
    let upperPoem = map toUpper poem
    -- IO write (output) stuff here
    putStr upperPoem

In [25]:
:type printUpper

printUpper :: FilePath -> IO ()

In [27]:
printUpper "john.vcard"

BEGIN:VCARD
VERSION:3.0
PRODID:-//APPLE INC.//ADDRESS BOOK 6.1.2//EN
N:DOE;JOHN;;;
FN:DOE JOHN
ORG:QUEEN MARY;
EMAIL;TYPE=INTERNET;TYPE=HOME;TYPE=PREF:JOHNDOE@NOGMAIL.COM
TEL;TYPE=CELL;TYPE=VOICE;TYPE=PREF:0751 234567
TEL;TYPE=HOME;TYPE=VOICE:020 7123 4567
ITEM1.ADR;TYPE=HOME;TYPE=PREF:;;42 NOWHERE ST;LONDON;;E1 0XX;
ITEM1.X-ABADR:GB
X-ABUID:85152BB5-BFB5-45DA-853A-BA021C7A0FC8:ABPERSON
END:VCARD

## Task 2. Do block structure

Do blocks contain a sequence of declarations: 
- IO binders: ``<name> <- <io_expression>``
- ordinary declarations ``let <name> = <expression>`` (note the `let`)
- expressions

They must end with an expression of IO type. If necessary use the return keyword to achieve this. 

Try this with and without the `return`:

In [29]:
do 
  poem <- readFile "poem.txt"
  let first_word = head $ words poem
  return first_word

"Twas"

Only the last `return` counts: 

In [31]:
do 
  poem <- readFile "poem.txt"
  let first_word = head $ words poem
  return first_word
  let second_word = head $ tail $ words poem
  return second_word

Line 1: Redundant return
Found:
do poem <- readFile "poem.txt"
   let first_word = head $ words poem
   return first_word
   let second_word = head $ tail $ words poem
   return second_word
Why not:
do poem <- readFile "poem.txt"
   let first_word = head $ words poem
   let second_word = head $ tail $ words poem
   return second_word

"brillig,"

There is a subtle point here. The do block above returns an `IO String` which is being executed and printed (we're returning a `String`). This on the other hand returns an `IO ()`, and just does the printing: 

In [32]:
do 
  poem <- readFile "poem.txt"
  let first_word = head $ words poem
  putStrLn first_word
  let second_word = head $ tail $ words poem
  putStrLn second_word

Twas
brillig,

**Exercise:** Write a simple word count program in the form of a function: 

```wc :: FilePath -> IO ()```

This should count the lines, words and characters of the relevant file, and print them with the name of the file. So 

``wc "poem.txt"`` 

should print: 

``poem.txt
lines: 35
words: 166
chars: 939``

Note: you may find the functions, `lines`, `words` and `length` helpful, as well as the function `show` which 
converts an element of a printable type into a String, so that you can print it out.  

In [40]:
wc filename = 
  do 
    text <- readFile filename
    let numLines = length $ lines text
    let numWords = length $ words text
    let numChars = length $ text
    putStrLn filename
    putStrLn $ "lines: "++(show numLines)
    putStrLn $ "words: "++(show numWords)
    putStrLn $ "chars: "++(show numChars)
    
wc "poem.txt"

Line 6: Redundant $
Found:
length $ text
Why not:
length textLine 8: Redundant bracket
Found:
"lines: " ++ (show numLines)
Why not:
"lines: " ++ show numLinesLine 9: Redundant bracket
Found:
"words: " ++ (show numWords)
Why not:
"words: " ++ show numWordsLine 10: Redundant bracket
Found:
"chars: " ++ (show numChars)
Why not:
"chars: " ++ show numChars

poem.txt
lines: 35
words: 166
chars: 939

`Eq` is the class of types supporting equality. 

`Ord` is the class of types supporting an order. 

`Show` is the class of types whose values can be printed (converted to `String`s). 

`Read` is the class of types whose values can be parsed from `String`s.

We've seen that the compiler will take any functions of the appropriate type to make instances of the typeclass. 
But often there are formal requirements couched in terms of expected equations, and informal requirements in terms 
of expectations. For simple datastructures, the compiler can derive a default instance of the standard typeclasses. 
It is instructed to do this using the `deriving` keyword. 

**EXERCISE** Here are two definitions of the Booleans, both using an enumerated type, and both deriving membership of typeclasses. What is the difference (and more importantly, how would it impact programs)? `T1` and `T2` represent `True`, and `F1` and `F2` represent `False`.

In [35]:
data B1 = F1 | T1 deriving (Eq, Ord, Show, Read) 

In [37]:
data B2  = T2 | F2 deriving (Eq, Ord, Show, Read) 

Explain difference here: 

## Task 3. Hierarchies

Sometimes we require that when we want to make a type a member of a typeclass, it should already be a memnber of some other. There is a hierarchy. 

Example: `Ord` requires that the type already be a member of `Eq`. 

```class Eq a => Ord a where ...```

**Exercise:** Make `PackString` an instance of `Eq` where two packed strings are regarded as equal if they have the same length. 

**Exercise:** Make `PackString` an instance of `Ord` where the ordering is that one string is shorter than the other. 

## Task 4. Deriving Show

**Exercise:** `PackStringB` is defined similarly to `PackString` but without deriving `Show`. Can you explain why you get an error when trying to evaluate `PackB "aaa"` but not `unpackB (packB "aaa")`?

In [41]:
data PackStringB = PackB String
unpackB (PackB s) = s

Line 1: Use newtype instead of data
Found:
data PackStringB = PackB String
Why not:
newtype PackStringB = PackB String

In [43]:
PackB "aaa"

: 

In [44]:
unpackB (PackB "aaa")

"aaa"

Explanation goes here.

## Task 5. Using and Defining Higher Order Functions